# Feta Brain ID

In [2]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import csv
import tensorflow as tf
from tensorflow.keras.models import Model
from neurite.tf import models  # Assuming the module's location
import voxelmorph.tf.losses as vtml
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
import neurite as ne
import sys
import nibabel as nib
tf.get_logger().setLevel('ERROR')
log_dir='logs_feta'
models_dir='models_feta'

import numpy as np
class CustomTensorBoard(tf.keras.callbacks.TensorBoard):
    def __init__(self, base_log_dir, **kwargs):
        super(CustomTensorBoard, self).__init__(**kwargs)
        self.base_log_dir = base_log_dir

    def on_epoch_begin(self, epoch, logs=None):
        if epoch % 200 == 0:  # Check if it's the start of a new set of 50 epochs
            self.log_dir = f"{self.base_log_dir}/epoch_{epoch}"
            super().set_model(self.model)




class PeriodicWeightsSaver(tf.keras.callbacks.Callback):
    def __init__(self, filepath, save_freq=200, **kwargs):
        super().__init__(**kwargs)
        self.filepath = filepath
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        # Save the weights every `save_freq` epochs
        if (epoch + 1) % self.save_freq == 0:
            weights_path = os.path.join(self.filepath, f"weights_epoch_{epoch + 1}.h5")
            self.model.save_weights(weights_path)
            print(f"Saved weights to {weights_path}")


weights_saver = PeriodicWeightsSaver(filepath=models_dir, save_freq=50)  # Save weights every 5 epochs

TB_callback = CustomTensorBoard(
    base_log_dir=log_dir,
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None
)

def dice_loss(y_true, y_pred):
    ndims = len(y_pred.get_shape().as_list()) - 2
    vol_axes = list(range(1, ndims + 1))

    top = 2 * tf.reduce_sum(y_true * y_pred, vol_axes)
    bottom = tf.reduce_sum(y_true + y_pred, vol_axes)

    div_no_nan = tf.math.divide_no_nan if hasattr(
        tf.math, 'divide_no_nan') else tf.div_no_nan  # pylint: disable=no-member
    dice = tf.reduce_mean(div_no_nan(top, bottom))
    return -dice
    
def dice_coefficient(y_true, y_pred):
    ndims = len(y_pred.get_shape().as_list()) - 2
    vol_axes = list(range(1, ndims + 1))

    top = 2 * tf.reduce_sum(y_true * y_pred, vol_axes)
    bottom = tf.reduce_sum(y_true + y_pred, vol_axes)

    div_no_nan = tf.math.divide_no_nan if hasattr(
    tf.math, 'divide_no_nan') else tf.div_no_nan  # pylint: disable=no-member
    dice = tf.reduce_mean(div_no_nan(top, bottom))
    return dice

2023-10-24 10:49:49.409563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Test adult brain modle on feta

In [3]:
nb_labels=2
dimx=256
dimy=256
nb_features=64
batch_size=8
warp_max=2   
warp_max=2.5
warp_min=.5
warp_blur_min=np.array([2, 4, 8])
warp_blur_max=warp_blur_min*2
bias_blur_min=np.array([2, 4, 8])
bias_blur_max=bias_blur_min*2
initial_lr=1e-4
lr = 1e-4
lr_lin = 1e-4
nb_levels=5
conv_size=3
num_epochs=40000

in_shape = (dimx,dimy)
data_dir = 'feta_2d/'
adults_models_directory = 'models.bi.zb.0.2'
# models_directory = 'models3'
# subject_dirs='feta_2d'
model_file='weights_epoch_10500.h5'
checkpoint_path='models_feta/weights_epoch_1000.h5'
subject_dirs = [d for d in os.listdir(data_dir) if d.startswith('sub-')]
# output_dir = 'feta_2d'

gen_arg = {
    'in_shape': in_shape,
    'labels_in': [i for i in range(25)],
    'labels_out': {i: 1 if i > 0 else 0 for i in range(25)},  # This line creates the desired mapping
    'warp_min': 0.01,
    'warp_max': 2,
    'zero_background': 0.2
}



def read_colors_from_csv(filepath):
    colors = []
    with open(filepath, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in spamreader:
            if len(row):
                tokens = row[0].split()
                if tokens[0].isnumeric():
                    colors.append((int(tokens[2])/255.,int(tokens[3])/255.,int(tokens[4])/255.))
    return colors
        
def load_colormap_from_file(filepath):
    colors = read_colors_from_csv(filepath)#, delimiter='     ')
    return colors

from tensorflow.keras.models import load_model

unet_model = models.unet(input_shape=(dimx, dimy, 1), nb_features=nb_features, nb_labels=nb_labels, nb_levels=nb_levels, conv_size=conv_size)
input_img = Input(shape=(dimx, dimy,1))


gen_model_1 = ne.models.labels_to_image_new(**gen_arg, id=1)
generated_img, y = gen_model_1(input_img)
unet_model = models.unet(input_shape=(dimx, dimy, 1), nb_features=nb_features, nb_labels=nb_labels, nb_levels=nb_levels, conv_size=conv_size)
segmentation = unet_model(generated_img)
combined_model = Model(inputs=input_img, outputs=segmentation)

combined_model.load_weights(os.path.join(adults_models_directory,model_file))
unet_model = models.unet(input_shape=(dimx, dimy, 1), nb_features=nb_features, nb_labels=nb_labels, nb_levels=nb_levels, conv_size=conv_size)
unet_model.set_weights(combined_model.get_weights())


seg_image_filenames = []
real_image_filenames = []
fetal_data = []  # List to store fetal images
fetal_segmentation_masks = []  # List to store fetal segmentation masks


for subject_dir in subject_dirs:
    subject_prefix = subject_dir

    # Search for the T2-weighted image and segmented brain image with flexible naming patterns
    t2w_path = None
    dseg_path = None
    # print(os.path.join(data_dir, subject_prefix))
    for filename in os.listdir(os.path.join(data_dir, subject_prefix)):
        if "_T2w.nii.gz" in filename:
            t2w_path = os.path.join(data_dir, subject_prefix, filename)
        elif "_dseg.nii.gz" in filename:
            dseg_path = os.path.join(data_dir, subject_prefix, filename)

    if t2w_path is None or dseg_path is None:
        print(f"Data not found for subject {subject_prefix}. Skipping...")
        continue

    # Append the filenames to the respective lists
    real_image_filenames.append(t2w_path)
    seg_image_filenames.append(dseg_path)
    
# # Load model weights
# combined_model.load_weights(os.path.join(models_directory, model_file))

# Create a list to store predictions
num_images_to_visualize = 3
dice_coefficients = []
y_preds=[]
# real_imgs
# Load and test images and segmentation masks
for i in range(len(real_image_filenames)):
    # Load the 2D image
    img_path = real_image_filenames[i]
    real_img = nib.load(img_path).get_fdata()

    # Load the segmentation mask
    seg_img_path = seg_image_filenames[i]
    seg_img = nib.load(seg_img_path).get_fdata()
    # unique_labels = np.unique(seg_img)
    
    # Print the number of unique labels
    # num_unique_labels = len(unique_labels)
    # print("Number of unique labels:", num_unique_labels)
    # Preprocess the image (you can apply normalization or other preprocessing here)
    min_value = np.min(real_img)
    max_value = np.max(real_img)
    real_img = (real_img - min_value) / (max_value - min_value)
    

    
    # Perform predictions using your unet_model
    # Ensure the input shape is correct for your model
    real_img = tf.expand_dims(real_img, axis=0)  # Shape becomes (1, 160, 192, 1)
    # print(real_img.shape)
    fetal_data.append(real_img)
    fetal_segmentation_masks.append(seg_img)

    y_pred = unet_model.predict(real_img, verbose=0)

    # Post-process the prediction as needed (e.g., thresholding, argmax, etc.)
    y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

    # Append the real image and prediction to lists
    # real_imgs.append(real_img)
    y_preds.append(y_pred)
    dice = dice_coefficient(seg_img, y_pred)
    dice_coefficients.append(dice)

# Perform any further analysis or visualization as needed
average_dice = np.mean(dice_coefficients)
print(f"Average Dice coefficient for all images: {average_dice}")


2023-10-24 10:49:56.211972: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1a:00.0, compute capability: 7.0
2023-10-24 10:49:56.212883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31141 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1c:00.0, compute capability: 7.0
2023-10-24 10:49:56.213556: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31141 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1d:00.0, compute capability: 7.0


using final_pred_activation softmax for unet
using final_pred_activation softmax for unet
using final_pred_activation softmax for unet


2023-10-24 10:50:05.883682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


Average Dice coefficient for all images: 0.7563523875950228


In [4]:
X_train, X_test, y_train, y_test = train_test_split(fetal_data, fetal_segmentation_masks, test_size=0.2, random_state=42)

# unet_model = models.unet(input_shape=(dimx, dimy, 1), nb_features=nb_features, nb_labels=nb_labels, nb_levels=nb_levels, conv_size=conv_size)
input_img = Input(shape=(dimx, dimy,1))

def my_generator(label_maps, batch_size=1, same_subj=False, flip=False):
    print(len(label_maps))
    in_shape = label_maps[0].shape
    num_dim = len(in_shape)
    void = np.zeros((batch_size, *in_shape), dtype='float32')
    rand = np.random.default_rng()
    prop = dict(replace=False, shuffle=False)
    num_batches = len(label_maps) // batch_size
    while True:
        ind = rand.integers(len(label_maps), size=2 * batch_size)
        x = [label_maps[i] for i in ind]
        if same_subj:
            x = x[:batch_size] * 2
        x = np.stack(x)[..., None]

        if flip:
            axes = rand.choice(num_dim, size=rand.integers(num_dim + 1), **prop)
            x = np.flip(x, axis=axes + 1)

        src = x[:batch_size, ...,0]
        y = np.array(void)
        yield src, y
        
# gen  = my_generator(input_img,batch_size=batch_size, same_subj=False, flip=False)
gen_arg = {
    'in_shape': in_shape,
    'labels_in': [i for i in range(8)],
    'labels_out': {i: 1 if i > 0 else 0 for i in range(8)},  # This line creates the desired mapping
    'warp_min': 0.01,
    'warp_max': 2.5,
    'zero_background': 0.2
}

gen_model_1 = ne.models.labels_to_image_new(**gen_arg, id=1)
generated_img, y = gen_model_1(input_img)
unet_model = models.unet(input_shape=(dimx, dimy, 1), nb_features=nb_features, nb_labels=nb_labels, nb_levels=nb_levels, conv_size=conv_size)
segmentation = unet_model(generated_img)
combined_model = Model(inputs=input_img, outputs=segmentation)

combined_model.load_weights(os.path.join(adults_models_directory,model_file))
unet_model = models.unet(input_shape=(dimx, dimy, 1), nb_features=nb_features, nb_labels=nb_labels, nb_levels=nb_levels, conv_size=conv_size)
unet_model.set_weights(combined_model.get_weights())


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # You can adjust the learning rate
# input_image=y_train
gen  = my_generator(y_train,batch_size=batch_size, same_subj=False, flip=False)
combined_model.add_loss(dice_loss(y, segmentation))
combined_model.compile(optimizer=optimizer)


if os.path.exists(checkpoint_path):
    combined_model.load_weights(checkpoint_path)
    print("Loaded weights from the checkpoint and continued training.")
else:
    print("Checkpoint file not found.")
    
hist = combined_model.fit(
    gen,
    epochs=num_epochs,  # Set the total number of epochs including previous ones
    initial_epoch=0,  # Specify the initial epoch
    verbose=0,
    steps_per_epoch=10,
    callbacks=[weights_saver, TB_callback]
)

using final_pred_activation softmax for unet
using final_pred_activation softmax for unet
Checkpoint file not found.
64


2023-10-24 10:50:26.152990: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4a01160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-24 10:50:26.153067: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-10-24 10:50:26.153082: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-10-24 10:50:26.153096: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-10-24 10:50:26.159525: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-24 10:50:26.317720: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proces

Saved weights to models_feta/weights_epoch_50.h5
Saved weights to models_feta/weights_epoch_100.h5
Saved weights to models_feta/weights_epoch_150.h5
Saved weights to models_feta/weights_epoch_200.h5
Saved weights to models_feta/weights_epoch_250.h5
Saved weights to models_feta/weights_epoch_300.h5
Saved weights to models_feta/weights_epoch_350.h5
Saved weights to models_feta/weights_epoch_400.h5
Saved weights to models_feta/weights_epoch_450.h5
Saved weights to models_feta/weights_epoch_500.h5
Saved weights to models_feta/weights_epoch_550.h5
Saved weights to models_feta/weights_epoch_600.h5
Saved weights to models_feta/weights_epoch_650.h5
Saved weights to models_feta/weights_epoch_700.h5
Saved weights to models_feta/weights_epoch_750.h5
Saved weights to models_feta/weights_epoch_800.h5
Saved weights to models_feta/weights_epoch_850.h5
Saved weights to models_feta/weights_epoch_900.h5
Saved weights to models_feta/weights_epoch_950.h5
Saved weights to models_feta/weights_epoch_1000.h5


KeyboardInterrupt: 